# CASE CAP GEMINI INVENT

**Objectifs de la partie 3**

L'objectif ici est simple: créer une pipeline de modèles de régression à entraîner puis tester leurs performances sur nos trois datasets que l'on nommera simplement rough_ml, by_hand et by_reg qui on le rappelle ont été obtenus sans sélection des variables, par sélection à la main et par elastic net.

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import explained_variance_score, r2_score, mean_absolute_error

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import clone
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor


In [74]:
rough_ml = pd.read_csv('df_ml/df_for_ml.csv')
by_hand = pd.read_csv('df_ml/ml_selected_byhand.csv')
by_reg = pd.read_csv('df_ml/ml_selected_byreg.csv')
unnorm_target = pd.read_csv('df_ml/unnormalized_target.csv')

df_dict = {'rough_ml':rough_ml, 'by_hand':by_hand, 'by_reg':by_reg}

## Description des modèles:
---

### Modèles de Régression Implementés 📈

#### Régression Linéaire Multiple 🧮
- **Description**: La régression linéaire multiple est un modèle statistique qui cherche à prédire une variable dépendante à partir de plusieurs variables indépendantes, en supposant une relation linéaire entre elles.
- **Utilisation**: Elle est idéale pour mettre en lumière des relations simples et pour servir de benchmark pour des 
modèles plus sophistiqués

#### Elastic Net 🕸️
- **Description**: Elastic Net est une méthode de régression régularisée qui combine les pénalités L1 et L2 des régularisations Lasso et Ridge. 
- **Utilisation**: Très utile lorsque plusieurs caractéristiques sont corrélées entre elles. Elastic Net peut aider à réduire le surapprentissage en introduisant ces pénalités.
- **Avantages**: Permet une sélection de caractéristiques automatique et la régularisation. Performant lorsque le nombre de prédicteurs est très grand.
- **Inconvénients**: Difficile de choisir ses paramètres.

#### Random Forest 🌳
- **Description**: Un modèle qui opère en construisant un grand nombre d'arbres de décision au moment de l'entraînement et en produisant la moyenne des prédictions de ces arbres pour la prédiction finale.
- **Utilisation**: Excellent pour capturer des relations non linéaires sans nécessiter une transformation manuelle des caractéristiques.
- **Avantages**: Réduit le risque de surapprentissage. Importances des caractéristiques facilement extractibles ce qui améliore l'"explicabilité" du modèle.
- **Inconvénients**: Moins interprétable qu'un modèle de régression linéaire.

#### XGBoost 🚀
- **Description**: XGBoost (Extreme Gradient Boosting) est un algorithme d'apprentissage ensembliste qui construit de manière itérative des arbres de décision de manière à minimiser une fonction de perte.
- **Utilisation**: Idéal pour des performances élevées et la rapidité sur des tâches de prédiction.
- **Avantages**: Très performant, capable de gérer des données de grande dimension et des relations non linéaires.
- **Inconvénients**: Peut être sujet au surapprentissage si les hyperparamètres ne sont pas bien choisis. Moins intuitif à comprendre et interpréter.

#### CatBoost 🐱🚀
- **Description**: CatBoost est un algorithme de boosting qui utilise des arbres de décision et est optimisé pour traiter efficacement les variables catégorielles comme l'état d'un bien immobilier etc...
- **Utilisation**: Particulièrement efficace pour les ensembles de données avec de nombreuses caractéristiques catégorielles.
- **Avantages**: Offre une bonne performance avec peu de paramétrage. Gère bien les données catégorielles sans prétraitement.
- **Inconvénients**: Encore et toujours un risque non négligeable de surapprentissage, comme pour tous les algorithmes de boosting.

---

## Construction de la Pipeline

In [75]:
y_mean = unnorm_target['valeur_fonciere'].mean(axis=0) 
y_std = unnorm_target['valeur_fonciere'].std(axis=0)
y_mean, y_std

(211288.86529576036, 133993.18394892054)

In [76]:

models_params = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Elastic Net': {
        'model': ElasticNet(),
        'params': {
            'alpha': [0.1, 1, 10],
            'l1_ratio': [0.1, 0.5, 0.9]
        }
    },
    'Random Forest': {
        'model': RandomForestRegressor(),
        'params': {
            'n_estimators': [10, 50, 100, 200],
            'max_depth': [None, 2, 3, 4]
        }
    },
    'XGBoost': {
        'model': XGBRegressor(),
        'params': {
            'n_estimators': [10, 50, 100, 200],
            'learning_rate': [0.01, 0.1, 0.5],
            'max_depth': [None, 2, 3, 4],
            'objective': ['reg:squarederror']
        }
    },
    #'CatBoost': {
    #    'model': CatBoostRegressor(verbose=0),  # verbose=0 empêche les spams
    #    'params': {
    #        'iterations': [10, 50, 100],
    #        'learning_rate': [0.01, 0.1, 0.5],
    #        'depth': [2, 3, 4],
    #        'loss_function': ['RMSE']
    #    }
    #}
}


<span style="color:red">**Note importante:**</span> 

Ici nous donnons comme métriques d'évaluation la MSE, le R2 score ainsi que le MAPE. Ce dernier peut sembler peu pertinent du fait que les étiquettes soient normalisées (et MAPE est TRES sensible au petites valeurs) ou que xgboost ne l'utilise pas dans notre exemple comme fonction de perte... Cependant nous allons tout de même calculer une version modifiée par nos soins de cette métrique pour satisfaire un double objectif: mettre en perspective la performance du modèle avec de nouvelles métriques, retourner une métrique interprétable pour le client.

Expliquons en quoi nous l'avons modifiée:
- La formule initiale 

`mape = np.mean(np.abs((y_pred - y_test) / y_test)) * 100`

- Sa version modifiée 

`modified_mape = np.mean(np.abs((y_test - y_pred) * y_std / (y_test*y_std + y_mean))) * 100`

Cette deuxième version multiplie l'écart de prédiction par la std réelle de y afin de rendre compte de ce que cet écart observé signifie réellement lorsque l'on considère les valeurs foncieres non normalisée (std est considérée ici comme un écart moyen de valeurs à leur moyenne) 

On divise par y_test*y_std + y_mean afin d'effectuer un rapport sur une variable 'dénormalisée'

In [77]:
# Initialisez best_models pour stocker à la fois le meilleur estimateur et le meilleur score pour chaque modèle
best_models = {name: {'model': None, 'score': {'mse': np.inf, 'r2': np.inf, 'mape': np.inf}, 'df': None} for name in models_params.keys()}

for df_name, df in df_dict.items():
    X = df.drop('valeur_fonciere', axis=1)
    y = df['valeur_fonciere'] # Trèèèèèèès important
    
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns
    categorical_features_indices = [i for i, col in enumerate(X.columns) if col in categorical_cols]

    X_encoded = pd.get_dummies(X, drop_first=True)
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

    print(50*'-', 'stats for : ', df_name, 50*'-')

    for name, mp in models_params.items():
        
        print(30*'-', name)

        model = clone(mp['model']) # On s'assure que la validation croisée commencera avec un modèle non entraîné

        if name == 'CatBoost':
            X_train, X_test, _, _ = train_test_split(X, y, test_size=0.2, random_state=42)
            model.set_params(**{'cat_features': categorical_features_indices})
            params = mp['params']
        else:
            params = mp['params']

        grid = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        grid.fit(X_train, y_train)
        score = grid.score(X_test, y_test)
        
        y_pred = grid.best_estimator_.predict(X_test)
        # Scores additionnels visant à avoir d'autres perspectives
        r2 = r2_score(y_test, y_pred)
        modified_mape = np.mean(np.abs((y_test - y_pred) * y_std / (y_test*y_std + y_mean))) * 100

        print(f"{name} MSE: {-score:.2f}")
        print(f"{name} R2: {r2:.2f}")
        print(f"{name} MAPE: {modified_mape:.2f}")
        
        best_models[name] = {'model': grid.best_estimator_, 
                                'score': {'mse': -score, 
                                        'r2': r2, 
                                        'mape': modified_mape}, 
                            'df': df_name}


-------------------------------------------------- stats for :  rough_ml --------------------------------------------------
------------------------------ Linear Regression


Linear Regression MSE: 273826880774065061888.00
Linear Regression R2: -262964839779657318400.00
Linear Regression MAPE: 311597388433.63
------------------------------ Elastic Net
Elastic Net MSE: 0.20
Elastic Net R2: 0.81
Elastic Net MAPE: 21.26
------------------------------ Random Forest
Random Forest MSE: 0.15
Random Forest R2: 0.85
Random Forest MAPE: 16.82
------------------------------ XGBoost
XGBoost MSE: 0.16
XGBoost R2: 0.84
XGBoost MAPE: 18.06
-------------------------------------------------- stats for :  by_hand --------------------------------------------------
------------------------------ Linear Regression
Linear Regression MSE: 0.18
Linear Regression R2: 0.83
Linear Regression MAPE: 21.64
------------------------------ Elastic Net
Elastic Net MSE: 0.20
Elastic Net R2: 0.80
Elastic Net MAPE: 21.28
------------------------------ Random Forest
Random Forest MSE: 0.16
Random Forest R2: 0.85
Random Forest MAPE: 16.83
------------------------------ XGBoost
XGBoost MSE: 0.16


Et là, nous sommes très déçus. Pourquoi? Parce que comme vu précédemment sur les variables sélectionnées, la valeur foncière dépend surtout de la surface du bâti, c'est même la feature jugée la plus importante par notre premier modèle catboost, ainsi que le prix moyen des voisins qui semble avoir une corrélation linéaire avec la valeur foncière... Ainsi nos modèles de boosting fonctionnent (sur les variables sélectionnées) pas bien mieux (du point de vue de la MSE) qu'une simple régression linéaire.

In [5]:
# On génère des données d'exemple
np.random.seed(0)
y = np.random.normal(0, 1, 1000)  

y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)

# On prend des prédictions y_pred qui sont 10% inférieures à y_test
y_pred = y_test * 0.9

y_mean = np.mean(y)
y_std = np.std(y)

# Testons notre métrique!
modified_mape = np.mean(np.abs((y_test - y_pred) * y_std / (y_test*y_std + y_mean))) * 100

modified_mape

13.033295341776352

Le MAPE modifié semble faire bon effet même s'il surestime un peu les écarts (ce qui n'est pas si mal)! 